# Modele językowe - n-gramy 

---

## 1. N-gramy słów w klasyfikacji
Poniżej stworzono kod, który przeprowadza klasyfikację dokumentów należących do 4 kategorii. W odróżnieniu do poprzednich zajęć - tu zaproponowano klasyfikator SVC (algorytm SVM, popularna alternatywa dla NaiveBayes), która również świetnie się spisuje w problemach klasyfikacji tekstu.

**<span style="color: red">Zadanie 1a (0.5 punktu)</span>** Uruchom kod, przyjrzyj się wygenerowanym wynikom (a najlepiej zachowaj je gdzieś, będą potrzebne). <br/>
Zapoznaj się z dokumentacją TfIdfVectorizer, odnajdź opcję uwzględnienia nie tylko pojedynczych słów jako cechy, ale także ich par i **zmodyfikuj poniższy kod tak, aby klasyfikacja uwzględniała zarówno pojedyncze słowa jak i pary (pozostaw parametr max_df=0.1 nienaruszony).** <span style="color: red"> Zmodyfikuj linię 30.</span><br/> <br/>
**<span style="color: red">Zadanie 1b (0.5 punktu)</span>** Jak zmieniła się liczba cech po uwzględnieniu tych par? Czy coś się zmieniło w raporcie z klasyfikacji? Uzupełnij odpowiedzi na pytania w komórce poniżej kodu.

In [1]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
# zbiór danych zawarty w Sklearn, który zawiera dane z 20 grup newsowych
from sklearn.datasets import fetch_20newsgroups
import numpy as np

# ------------------- WCZYTANIE DANYCH -----------
# ustaw seed na 0, aby zapewnić powtarzalność eksperymentu
np.random.seed(0)

# kategorie do analizy
categories = ['sci.space', 'comp.graphics', 'talk.politics.misc', 'comp.sys.mac.hardware']

# pobieramy zbiór uczący/testowy (na nim będziemy trenować) dla wybranych kategorii.
train = fetch_20newsgroups(subset='train', categories=categories)
test = fetch_20newsgroups(subset='test', categories=categories)

# ------------------- STWORZENIE PIPELINE'U -----------
# stwórzmy pipeline surowy tekst -> TFIDF vectorizer -> klasyfikator
pipeline = Pipeline([
  ('tfidf', TfidfVectorizer(max_df=0.1, ngram_range=(1, 2))),
  ('clf', SVC(C=1.0, kernel='linear')),
])

# ------------------- TRANSFORMACJA I UCZENIE -----------
# zwektoryzujmy dane i wytrenujmy klasyfikator na zbiorze treningowym
pipeline.fit(train.data, train.target)

print(f"W słowniku znajduje się {len(pipeline.named_steps['tfidf'].vocabulary_.keys())} różnych cech")

# ------------------- OCENA KLASYFIKATORA -----------
# testowanie klasyfikatora - szerokie podsumowanie uwzględniające miary:
#  - precision,
#  - recall,
#  - f1
print(classification_report(test.target, pipeline.predict(test.data)))

W słowniku znajduje się 287718 różnych cech
              precision    recall  f1-score   support

           0       0.87      0.94      0.91       389
           1       0.92      0.94      0.93       385
           2       0.95      0.90      0.92       394
           3       0.97      0.90      0.93       310

    accuracy                           0.92      1478
   macro avg       0.93      0.92      0.92      1478
weighted avg       0.92      0.92      0.92      1478



<pre>
W słowniku znajduje się 34774 różnych cech
              precision    recall  f1-score   support

           0       0.88      0.93      0.90       389
           1       0.91      0.93      0.92       385
           2       0.94      0.91      0.92       394
           3       0.96      0.90      0.93       310

    accuracy                           0.92      1478
   macro_avg       0.92      0.92      0.92      1478
weighted_avg       0.92      0.92      0.92      1478
</pre>
<pre>
W słowniku znajduje się 287718 różnych cech
              precision    recall  f1-score   support

           0       0.87      0.94      0.91       389
           1       0.92      0.94      0.93       385
           2       0.95      0.90      0.92       394
           3       0.97      0.90      0.93       310

    accuracy                           0.92      1478
   macro avg       0.93      0.92      0.92      1478
weighted avg       0.92      0.92      0.92      1478
</pre>

1. O ile zwiększyła się liczba cech w klasyfikatorze?
2. Czy precyzja w którejkolwiek klasie wzrosła? w której/których?
3. Czy recall w którejkolwiek klasie wzrósł? w której/których?

In [2]:
print(f"1. Liczba cech w klasyfikatorze zwiększyła się o: {abs(287718 - 34774)}")
print(f"2. Precyzja w którejkolwiek klasie wzrosła w klasyfikatorze zawierającym unigramy i bigramy")
print(f"3. Recall lekko zwiększył się w klasyfikatorze zawierającym unigramy i bigramy")

1. Liczba cech w klasyfikatorze zwiększyła się o: 252944
2. Precyzja w którejkolwiek klasie wzrosła w klasyfikatorze zawierającym unigramy i bigramy
3. Recall lekko zwiększył się w klasyfikatorze zawierającym unigramy i bigramy


## 2. N-gramy liter w klasyfikacji
Poza n-gramami stworzonymi z następujących po sobie wyrazów - bardzo często używane są również n-gramy znakowe, stworzone z następujących po sobie liter. <br/><br/>
Dla przykładu. wszystkie 3-gramy (trigramy) znakowe z napisu "Hello world" to: <br/>
"Hel", "ell", "llo", "lo ", "o w", " wo", "wor", "orl", "rld". <br/><br/>
Do czego mogłaby być użyta taka reprezentacja tekstów? Okazuje się, że całkiem mocno pomaga to w rozwiązywaniiu problemu detekcji języka w którym został zapisany dokument, szczególnie w sytuacji, kiedy teksty są bardzo krótkie (np. tweety, smsy).
<br/>
Poniżej znajduje się szkielet klasyfikatora rozpoznającego język w którym zapisany jest dokument.
Języków jest 6: polski, angielski, niemiecki, francuski, hiszpański i włoski.
<br/>
**<span style="color: red">Zadanie 2 (1 punkt)</span>**: Przedstawiony klasyfikator jest znanym już z poprzednich przykładów kodem. Waszym zadaniem jest:
<ol>
    <li>Zapoznanie się dokumentacją Tf-Idf vectorizera, aby znaleźć funkcjonalność, która zamiast całych słów, stworzy cechy na podstawie liter i wykorzystanie tej funkcjonalności w kodzie</li>
    <li>Ustawienie takiego zakresu n-gramów, aby zmaksymalizować uzyskany wynik (Oczekiwane 1.0 precyzji i recallu we wszystkich kategoriach przy pozostawieniu wartośi max_features = 300 elementów)</li>
    <li>Poprawnie zaklasyfikuje krotki przykład zapisany w linii 43 (Bonjour przypisze do kategorii 'french').</li>
</ol>

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import pandas
import numpy as np

# -------------------- FUNKCJE POMOCNICZE --------

# Funkcja mapująca identyfikator liczbowy kategorii na wartość tekstową, np: 0->"polish", 1->"english"
def get_class_name_from_id(ids, mapping): return [mapping[id_] for id_ in ids]
# ------------------- WCZYTANIE DANYCH -----------
# wczytaj dane z pliku CSV
full_dataset = pandas.read_csv('resources/language-detection-1000.csv', encoding='utf-8')
lang_to_id = {'polish': 0, 'english': 1, 'french': 2, 'german': 3, 'italian': 4, 'spanish': 5}
id_to_lang = {v: k for k, v in lang_to_id.items()}
# Ponieważ nazwy kategorii zapisane są z użyciem stringów: "ham"/"spam",
# wykonujemy mapowanie tych wartości na liczby, aby móc wykonać klasyfikację.
full_dataset['label_num'] = full_dataset.lang.map(lang_to_id)

# ustaw seed na 0, aby zapewnić powtarzalność eksperymentu
np.random.seed(0)
# wylosuj 70% wierszy, które znajdą się w zbiorze treningowym
train_indices = np.random.rand(len(full_dataset)) < 0.7

# Wybierz zbiór treningowy (70%)
train = full_dataset[train_indices]
# Wybierz zbiór testowy (dopełnienie treningowego - 30%)
test = full_dataset[~train_indices]

# ------------------- STWORZENIE PIPELINE'U -----------
# stwórzmy pipeline surowy tekst -> TFIDF vectorizer -> klasyfikator
pipeline = Pipeline([
  ('tfidf', TfidfVectorizer(max_features=300, analyzer='char_wb', ngram_range=(1, 5))),
  ('scaler', StandardScaler(with_mean=False)),
  ('clf', LogisticRegression()),
])
# ------------------- TRANSFORMACJA I UCZENIE -----------

# zwektoryzujmy dane i wytrenujmy klasyfikator na zbiorze treningowym
pipeline.fit(train['text'], train['label_num'])

print("Oto kilka przykładowych cech stworzonych przez TfidfVectorizer:"
      f" {list(pipeline.named_steps['tfidf'].vocabulary_.keys())[:5]}")

# ------------------- WERYFIKACJA NA KRÓTKIM TEKŚCIE ----
text_to_predict = "Bonjour!"
predicted = pipeline.predict([text_to_predict])
print(f"\n\nTekst: {text_to_predict} został zaklasyfikowany jako: {id_to_lang[predicted[0]]}\n\n")

# ------------------- OCENA KLASYFIKATORA -----------
print(classification_report(
  get_class_name_from_id(test['label_num'], id_to_lang),
  get_class_name_from_id(pipeline.predict(test['text']), id_to_lang)
))

c:\users\hououin kyouma\pycharmprojects\nlp-laboratories\venv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Oto kilka przykładowych cech stworzonych przez TfidfVectorizer: [' ', 'a', 'p', 'e', 'l']


Tekst: Bonjour! został zaklasyfikowany jako: french


              precision    recall  f1-score   support

     english       1.00      0.99      1.00       303
      french       1.00      1.00      1.00       280
      german       0.99      1.00      1.00       337
     italian       1.00      1.00      1.00       273
      polish       1.00      1.00      1.00       291
     spanish       1.00      1.00      1.00       299

    accuracy                           1.00      1783
   macro avg       1.00      1.00      1.00      1783
weighted avg       1.00      1.00      1.00      1783



Widzimy, że problem jest stosunkowo prosty. Po co zatem używać n-gramów znakowych? Aby zaoszczędzić pamięć i podołać sytuacjom, w których zbiór testowy składa się ze słów, które nie występują w korpusie uczącym. <br/>

O ile wszystkich słów w tych 6 językach jest "30078", to trigramów znakowych jest już tylko "15274", a bigramów - "2059". W związku z tym: <ol>
<li>Używając n-gramów znakowych często możemy ograniczyć liczbę cech</li>
<li>N-gramy znakowe pomogą nam w sytuacjach, kiedy dane słowo nie wystąpiło w tekście uczącym. Jeśli opieramy uczenie na pełnych słowach i cały nasz tekst testowy to niewystępujące w korpusie uczącym - "bonjour", wtedy wektor BOW będzie zawierał same zera, przez co będzie miał problem z przydziałem do odpowiedniej klasy. <br/> N-gramy znakowe nawet jeśli nie napotkały danego słowa podczas analizy korpusu, to na podstawie budowy samego słowa są w stanie przewidywać do jakiego języka słowo należy. Np. cokolwiek zawierającego trigram "żeb" należeć będzie raczej do języka polskiego.</li>
</ol>

---

### 2a - istotność cech

Ponieważ w zadaniu 2 użyliśmy znanego z zajęć z przedmiotu "Sztuczna Inteligencja" klasyfikatora liniowego - regresji logistycznej, podejrzeć możemy jakie cechy najsilniej sugerują nam przynależność do danej klasy. Uruchom poniższy kod, aby zobaczyć jakie cechy są najważniejsze dla danych kategorii. Modyfikując parametry TfidfVectorizer możesz zobaczyć jakie słowa/ciągi znaków są najistotniejsze do detekcji danego języka.

In [4]:
# Funkcja, z której użyciem możemy ocenić które cechy najsilniej skojarzone są z danymi klasami.
# Wyświetli listę słów/n-gramów znakowych, których obecność najsilniej wpływa na przydział do danej klasy
def language_indicators(feature_names, feature_importances, id_to_lang):
  # iterujemy po macierzy feature_importances (wymiarów: język x cechy) wierszami (czyli język po języku)
  for i, language in enumerate(feature_importances):
    # Tworzymy skojarzenie nazw cech z wagami modelu
    # (ponieważ używamy regresji logistycznej — każda cecha (słowo/n-gram)
    # ma swoją wagę, która jest optymalizowana w procesie uczenia!
    # Cechy z wysokimi wagami są ważne dla danej klasy.
    # Każda klasa ma osobny model ze swoimi wagami!)
    # posortujmy cechy skojarzone z wagami malejąco
    scored_features = sorted(zip(feature_names, language), key=lambda x: x[1], reverse=True)
    #zamieńmy identyfikator numeryczny kategorii na nazwę języka
    print(f"W rozpoznaniu języka {id_to_lang[i]} najważniejsze cechy to:")
    # wybierzmy po 5 najważniejszych cech (najwyższe wartości uczonych współczynników)
    for feature, score in scored_features[:5]:
      print(f"\t'{feature}': {score}")

# ------------------- WYŚWIETLENIE NAJWAŻNIEJSZYCH CECH DLA KAŻDEJ KATEGORII
language_indicators(
  # Pobierz nazwy cech
  pipeline.named_steps['tfidf'].get_feature_names(),
  # Pobierz wyuczone współczynniki
  # (regresja logistyczna to stworzenie modelu opisanego wzorem y = e^(-wx - b),
  # gdzie uczymy się współczynników w.
  # Pole coef_ zawiera te współczynniki dla każdego języka z osobna)
  pipeline.named_steps['clf'].coef_,
  # mapowanie z identyfikatora numerycznego na pełną nazwę języka — zwiększa czytelność wygenerowanego raportu
  id_to_lang
)

W rozpoznaniu języka polish najważniejsze cechy to:
	'j': 0.19901617632544774
	'ł': 0.18816129382371247
	'ę': 0.18320232502803763
	'k': 0.17359664559466248
	'y': 0.17051681188204693
W rozpoznaniu języka english najważniejsze cechy to:
	'th': 0.23285452718562907
	' an': 0.22631352214163009
	' th': 0.22288205475537914
	'the': 0.2158537554985106
	'y ': 0.21250378502730857
W rozpoznaniu języka french najważniejsze cechy to:
	'é': 0.3959984284874838
	''': 0.23530640345092413
	'ce': 0.23522657533847321
	't ': 0.22677827203788822
	'ou': 0.22428070168981273
W rozpoznaniu języka german najważniejsze cechy to:
	'ei': 0.20423099191463112
	'der': 0.17450101370170895
	'en ': 0.17434925387262365
	'en': 0.1740649213430604
	' au': 0.16620582946503853
W rozpoznaniu języka italian najważniejsze cechy to:
	'i ': 0.34884806893525017
	'o ': 0.2801741071699971
	'tt': 0.20436243361062018
	'zi': 0.20107410278801147
	'i': 0.2003319918014233
W rozpoznaniu języka spanish najważniejsze cechy to:
	'os ': 0.2921648

c:\users\hououin kyouma\pycharmprojects\nlp-laboratories\venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


---

## 3. N-gramy słów w generowaniu tekstu

Innym, bardzo ciekawym zastosowaniem n-gramów jest możliwość generowania tekstu z użyciem tzw. łańcuchów Markova. Stwórzmy funkcję generującą n-gramy słów, aby później móc ją wykorzystać do tworzenia tekstów.

**<span style="color: red">Zadanie 3 (1 punkt)</span>** stwórz funkcję, która wygeneruje n-gramy słów zadanego stopnia n (n_gram_len). Aby podzielić zdanie na słowa nie musisz używać tokenizatora z biblioteki, na potrzeby zadania wystarczy uznać, że spacja oddziela poszczególne słowa.
<br/>
<br/>
<div class="alert alert-success">
Oczekiwany rezultat dla zadanych danych: <br/><br/>[['The', 'big', 'brown'], ['big', 'brown', 'fox'], ['brown', 'fox', 'jumped'], ['fox', 'jumped', 'over'], ['jumped', 'over', 'the'], ['over', 'the', 'fence.']]
</div>
<br/>

In [28]:
from typing import Iterable
from typing import Literal

def generate_ngrams(sentence: str, n: int, mode: Literal['word', 'char']) -> Iterable[list[str] | str]:
  match mode:
    case 'word':
      items = sentence.split()
      return (items[i:i + n] for i in range(len(items) - n + 1))
    case 'char':
      items = list(sentence)
      return (''.join(items[i:i + n]) for i in range(len(items) - n + 1))
list(generate_ngrams("The big brown fox jumped over the fence.", 3, mode='word'))

[['The', 'big', 'brown'],
 ['big', 'brown', 'fox'],
 ['brown', 'fox', 'jumped'],
 ['fox', 'jumped', 'over'],
 ['jumped', 'over', 'the'],
 ['over', 'the', 'fence.']]

Jeśli udało Ci się napisać funkcję get_word_ngrams — zapoznaj się z poniższym kodem i uruchom go, aby wytworzyć tekst!

In [82]:
from collections import Counter, defaultdict

def generate_ngram_markov(text: str, n: int, mode: Literal['word', 'char']) -> dict[str, Counter]:
  def pair_context_last_word(items: list[str]) -> tuple[str, str]:
    (*items, last) = items
    match mode:
      case 'word':
        return (' '.join(items), last)
      case 'char':
        return (''.join(items), last)
  markov_dict = defaultdict(list[str])

  for (context, word) in map(pair_context_last_word, generate_ngrams(text, n, mode)):
    markov_dict[context].append(word)

  for context in markov_dict:
    markov_dict[context] = Counter(markov_dict[context])
  return markov_dict

n_gram_len = 3
with open("resources/polish-europarl.txt", 'r', encoding='utf-8') as file:
  text = file.read().lower()
markov_dict = generate_ngram_markov(text, n_gram_len, mode='word')
print(len(markov_dict.keys()))

274320


In [44]:
from random import randrange
from itertools import islice

def generate_sentence(start: str, n: int, appendices: int) -> str:
  for _ in range(appendices):
    context = " ".join(start.split(" ")[-n + 1:])
    # sprawdźmy słowa, które są dozwolone jako następniki naszego kontekstu (context)
    # i wybierzmy taki następnik,
    # który zostanie wylosowany zgodnie z rozkładem stworzonym przez histogram.
    index = randrange(sum(markov_dict[context].values()))
    new_word = next(islice(markov_dict[context].elements(), index, None))
    if not new_word: return start
    start += f" {new_word}"
  return start
print(generate_sentence('Średnio co dwa', n_gram_len, 500))

Średnio co dwa miesiące od początku roku wyraźnie wypowiadaliśmy nasze obawy co do dalszych cięć bez jakichkolwiek planów inwestycyjnych z krajami trzecimi, a to polega na zapewnieniu, by rynek pracy jest bardzo niski, a stopień wykorzystania środków ue. ponieważ jest on uprawniony do decydowania o statusie, prawach i strukturach. centralizacja polegająca na połączeniu badań i innowacji w o wiele za mało. nie możemy realizować tego rodzaju usług. czy europa coś z systemem handlu uprawnieniami do emisji, by zagwarantować zgodność pierwszego europejskiego programu doprowadzi do pogorszenia warunków życia i zwiększający konsumpcję. unia innowacji może pomóc zwiększyć zdolność ue do przeglądu wydatków we wszystkich państwach członkowskich. jestem przekonany, że przystąpienie rumunii i bułgarii do obszaru swobodnego przemieszczania się w porównaniu do wszystkich jego aspektów. oto dlaczego naszym celem powinno być żadnych nieporozumień. poruszymy tę kwestię, będąc w strefie euro i pozostały

---

## 4. N-gramy znakowe w generowaniu tekstu

W bardzo podobny sposób do zadania 3, możemy stworzyć model, który generować będzie tekst literka po literce. <br/>
**<span style="color: red">Zadanie 4 (1 punkt)</span>** stwórz funkcję, która wygeneruje n-gramy znakowe zadanego stopnia n (n_gram_len).
<br/>
<br/>
<div class="alert alert-success">
Oczekiwany rezultat dla zadanych danych: ['The', 'he ', 'e b', ' bi', 'big', 'ig ', 'g b', ' br', 'bro', 'row', 'own', 'wn ', 'n f', ' fo', 'fox', 'ox ', 'x j', ' ju', 'jum', 'ump', 'mpe', 'ped', 'ed ', 'd o', ' ov', 'ove', 'ver', 'er ', 'r t', ' th', 'the', 'he ', 'e f', ' fe', 'fen', 'enc', 'nce', 'ce.']
</div>
<br/>

In [48]:
# Funkcja zdefiniowana wcześniej
print(list(generate_ngrams("The big brown fox jumped over the fence.", 3, mode='char')))

['The', 'he ', 'e b', ' bi', 'big', 'ig ', 'g b', ' br', 'bro', 'row', 'own', 'wn ', 'n f', ' fo', 'fox', 'ox ', 'x j', ' ju', 'jum', 'ump', 'mpe', 'ped', 'ed ', 'd o', ' ov', 'ove', 'ver', 'er ', 'r t', ' th', 'the', 'he ', 'e f', ' fe', 'fen', 'enc', 'nce', 'ce.']


Po stworzeniu funkcji **get_character_ngrams()** możemy uruchomić generator znakowy.

In [143]:
n_gram_len = len(text)
with open('resources/mister-theodore.txt', 'r', encoding='utf-8') as file:
  text = file.read().lower()
markov_dict = generate_ngram_markov(text, n_gram_len, mode='char')

In [155]:
text = 'U szlachty'
for _ in range(500):
  context = text[-n_gram_len + 1:]
  index = randrange(sum(markov_dict[context].values()))
  text += next(islice(markov_dict[context].elements(), index, None))
print(text)

U szlachty «kropić, kropić!»
«ależ, najsłodszy jezu! trzeba raz rzecz skończy się do niej przez okiennic szpary,
i zgasło. i wnet sierpy gromadnie dzwoniąc w tabakierę palcami chrząsnął,
jak węgorz do odarcia: lecz nam *urodzonym*,
nam wielmożnym, do złotych swobód wzwyczajonym!
ach, bracie protazeńku» rzekł klucznik był podobny rysiowi rannemu,
który dziś żółty, dawniej na dwory pańskie, lub na łowy.
podczas uczty na chorze tym kapela stała,
i w organ, i w rozlicznej barwy *surojadki* srebrzystych na pró
U szlachty sąsiedzkiej gromada
za gościnnymi stoły sędziego zakazy,
w niebytność maćka zwykle usta do milczenie miał słuch bardzo czuły.
sam gawęda, i lubił niezmiernego słońca
i odbita o ciemne murawy wezgłowiu cieni
jeszcze zbyt wcześniej wstawszy, piły kawę;
teraz drugą dla siebie.

    i to wiadomo czemu.
zaczęli proces w ziemstwie i guberskim rządzie żyjesz, jesteś zuch na szpady: wyjdź ty bracie ryków.
lub wiesz co, wyszlem kogo z nich opis zwycięstwie,
prawie wszyscy — mieć na 

---

## 5. Ngramy do generowania tekstu - długość ngramu a jakość tekstu
<span style="color: red">**Zadanie 5 (1 punkt)**</span>
Obswerując wyniki z zadań 3 i 4 i sprawdzając różne długości n-gramów (znakowych i słów) zastanów się:
<ol>
<li>Jakie ryzyko w kontekście jakości tekstu niesie ze sobą tworzenie tekstu z bardzo **krótkich** n-gramów?</li>
<li>Jakie ryzyko w kontekście jakości tekstu niesie ze sobą tworzenie tekstu z bardzo **długich** n-gramów?</li>
</ol>
Odpowiedzi zawrzyj w komentarzu poniżej

In [ ]:
# Zad 5:
#  Pytanie 1: Wygenerowane teksty z krótkich ngramów bardzo łatwo zapominają o kontekście, przez co tracą spójność
#  Pytanie 2: Wygenerowane teksty z długich ngramów mogą nie znaleźć godnego następnika, przez co mają prawdopodobieństwo równe 0

---

## 6. Bonus: Prawdopodobieństwo wystąpienia zdania - bez punktów
Dodatkowo ciekawym zastosowaniem n-gramów jest również ocena - jak bardzo prawdopodobnym jest wystąpienie danego zdania w rzeczywistości. Kiedy rozwiązujemy zadanie translacji mowy na tekst, spotykamy się z sytuacjami, w których nie do końca wiemy, czy słowo, które zostało wypowiedziane to np. "morze" czy "może". Model językowy oparty o n-gramy może ocenić szansę wystąpienia danego ciągu wyrazów, a więc również wybrać bardziej prawdopodobny ciąg wyrazów w danym języku. <br/>

Biorąc pod uwagę, że zdanie to ciąg wyrazów    $w_1, w_2, w_3, ..., w_n$
Możemy poczynić upraszczające założenie, że aktualne słowo zależne jest jedynie od słowa poprzedniego, zatem prawdopodobieństwo wystąpienia zdania $P(sentence) = p(w_1|beginOfSentence)*p(w_2|w_1)*p(w_3|w_2)*...*p(w_n|w_(n-1))$

Obliczając prawdopodobieństwa warunkowe, może się okazać, że w testowanym przez nas zdaniu mogą wystąpić dwie problematyczne sytuacje:
<ol>
    <li>słowo konteksowe $w_c$ ze wzoru $p(w_n|w_c)$ nie występuje w korpusie - bardzo mała szansa jeśli korpus jest wystarczająco duży</li>
    <li>słowo następujące po kontekstowym ($w_n$) nie współwystępuje w korpusie ze słowem kontekstowym (więc $p(w_n|w_c) = 0$ - całkiem możliwy stan, dość łatwo można sobie wyobrazić sytuację braku współwystępowania pewnych słów nawet przetwarzając duży korpus</li>
</ol>

Aby poradzić sobie z sytuacją, w której chcemy aby pewne słowo rozpoczynało/kończyło tekst, możemy dodać sztuczne znaczniki początku (BOS - Begin of Sentence) i końca (EOS - End of Sentence) zdania. Wprowadzając te znaczniki, będziemy mogli obliczyć prawdopodobieństwo wystąpienia słowa, pod warunkiem, że rozpoczyna ono zdanie $p(w_n|BOS)$

Poniżej znajduje się kod oceniający prawdopodobieństwo wystąpienia zdań. Widzimy, że jedno z tych zdań ma sensowniejszy tekst i chcielibyśmy, aby komputer był w stanie wybrać sensowniejszą opcję.

Problematyczne sytuacje rozwiązane zostały następująco:
<ol>
<li>Jeśli brak słowa kontekstowego w wygenerowanym modelu - uznaj, że prawdopodobieństwo całego zania wynosi 0</li>
<li>Jeśli słowo następujące po kontekstowym nie współwystępuje z kontekstowym - użyj wygładzania aby ustawić prawdopodobieństwo na bardzo małą wartość (ale niezerową)</li>
</ol>

**Zapoznaj się z kodem i urochom go, tutaj nie trzeba nic zmieniać, to jedynie wizualizacja zastosowania. Uwaga - aby poprawnie oszacować prawdopodobieństwa potrzeba posiadać wykonane zadanie 3 (stworzona funkcja get_word_ngrams)**


In [ ]:
# tekst do oceny
text1 = "i heard that the european union is a valuable concept."
# tekst do oceny
text2 = "i had that the euro bean union is a variable concept."

# będziemy dzielić na zdania
from nltk import sent_tokenize
# i czyścić tekst
import re

from collections import Counter, defaultdict
import random
import itertools

# słownik zawierający częstości występowania słów w zależności od poprzedzającego je słowa
markov_dict = defaultdict(list)

def clean_text(text):
  # czyszczenie tekstu ze znaków nowej linii, tabulatorów, spacji (wielokrotnych)
  return re.sub("[\n\t ]+", " ", text)

def make_begin_end_of_sentences(text):
  result = ""
  sentences = sent_tokenize(text)
  for sent in sentences:
    # dla każdego zdania dodajemy specjalne tagi <BOS> = begin of sentence oraz <EOS> - end of sentence
    result += " <BOS> {s} <EOS> ".format(s=sent)
  return clean_text(result)

def get_sentence_probability(sentence, markov_dict):
  sentence = " <BOS> {s} <EOS> ".format(s=sentence)
  sentence = clean_text(sentence)

  sentence = sentence.split(' ')
  prob = 1.0
  for i in range(len(sentence)):
    if i < 1:
      continue

    # słowo poprzedzające
    context = sentence[i - 1]
    # aktualne słowo
    word = sentence[i]

    # jeśli słowo kontekstowe występuje w modelu - OK
    if context in markov_dict.keys():
      # jeśli słowo 'word' współwystępowało z
      # 'context' w korpusie - obliczmy prawdopodobieństwo tej sytuacji p(wn|wc)
      if word in markov_dict[context].keys():
        prob *= 1.0 * markov_dict[context][word] / sum(markov_dict[context].values())
      else:
        # smoothing, jeśli dane slowo 'word' nie występowało
        # po słowie 'context' w korpusie,
        # ustalmy wartość prawdopodobieństwa na bardzo neiwielką.
        prob *= 1 / (sum(markov_dict[context].values()) + 1)
  return prob


with open('english_europarl.txt', 'r') as f:
  text = clean_text(f.read().lower())
  text = make_begin_end_of_sentences(text)

  # wygeneruj wszystkie 2-gramy słów z korpusu
  n_grams = generate_ngrams(text, 2)
  # dla każdego n-gramu...
  for n_gram in n_grams:
    # weź przedostatnie słowo jako kontekst
    context = n_gram[-2]
    # weź ostatnie słowo jako kontekst
    last_word = n_gram[-1]
    # dopiszmy następniki, które występują w korpusie po kontekście
    markov_dict[context].append(last_word)

    # dla każdego kontekstu
  for context in markov_dict.keys():
    # stwórz histogram słów jakie występują w korpusie po tym kontekście
    markov_dict[context] = Counter(markov_dict[context])

  # wyznacz prawdopodobieństwo wystąpienia text1
  probability_of_sent1 = get_sentence_probability(text1, markov_dict)
  # wyznacz prawdopodobieństwo wystąpienia text2
  probability_of_sent2 = get_sentence_probability(text2, markov_dict)

  print("Prawdopodobieństwo wystąpienia zdania 1: {p}".format(p=get_sentence_probability(text1, markov_dict)))
  print("Prawdopodobieństwo wystąpienia zdania 2: {p}".format(p=get_sentence_probability(text2, markov_dict)))